In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
train_dataset=pd.read_csv("1traind.csv")
test_dataset = pd.read_csv("1testd.csv")

In [ ]:
train_dataset.shape,test_dataset.shape

In [ ]:
train_dataset.tail(5)

In [ ]:
test_dataset.head()

In [ ]:
print(train_dataset['label'].value_counts(),test_dataset['label'].value_counts(),sep="\n")

In [ ]:
from autogluon.multimodal import MultiModalPredictor

In [ ]:
# loaded_predictor = MultiModalPredictor.load('/DATA/jimson/Rishi/Priyadrasta/papercode/first floor/vgg16/ag-20230105_051955')
loaded_predictor = MultiModalPredictor(label="label")
loaded_predictor.fit(
    train_data=train_dataset, time_limit=3600, # seconds
    hyperparameters={"model.names":  ['timm_image'],"env.num_gpus": 1,"optimization.optim_type": "adam", 
    "optimization.max_epochs":30,"model.timm_image.checkpoint_name":'vgg19'
    }
)

In [ ]:
scores = loaded_predictor.evaluate(test_dataset, metrics=["accuracy"])
print('Top-1 test acc: %.3f' % scores["accuracy"])

In [ ]:
roc_auc = loaded_predictor.evaluate(test_dataset)
# print('Top-1 test acc: %.3f' % scores["roc_auc"])
roc_auc

In [ ]:
predictions = loaded_predictor.predict(test_dataset)
print(predictions)

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix
conf_matrix= confusion_matrix(test_dataset['label'].to_numpy(),predictions.to_numpy())
conf_matrix

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
 
fig, ax = plot_confusion_matrix(conf_mat=conf_matrix, figsize=(4, 4), cmap=plt.cm.Blues)
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix) 
FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
TP = np.diag(conf_matrix)
TN = conf_matrix.sum() - (FP + FN + TP)

In [ ]:
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

In [ ]:
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# Overall accuracy for each class
ACC = (TP+TN)/(TP+FP+FN+TN)

In [ ]:
print("class-wise  Accuracy :",ACC)
print("class-wise  Precision :",PPV)
print("class-wise  Sensitivity :",TPR)
print("class-wise  Specificity :",TNR)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_dataset['label'].to_numpy(), predictions.to_numpy()))